In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
from tensorflow.keras import models
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [2]:
# Configure the CNN (Convolutional Neural Network).
classifier = Sequential()
# Convolution - extracting appropriate features from the input image.
# Non-Linearity (RELU) - replacing all negative pixel values in feature map by zero.
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3),
               activation='relu'))
# Pooling: reduces dimensionality of the feature maps but keeps the most important information.
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Adding a second convolutional layer and flattening in order to arrange 3D volumes into a 1D vector.
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())
# Fully connected layers: ensures connections to all activations in the previous layer.
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=1, activation='sigmoid'))

In [17]:
from datetime import datetime
#tensorboard stuff
#logdir = "./FolderFrames/logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
#tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


# Compile the CNN and train the classifier..
classifier.compile(optimizer='adam', loss='binary_crossentropy',
                   metrics=['accuracy'])

train_imagedata = ImageDataGenerator(rescale=1. / 255, shear_range=0.2,
        zoom_range=0.2, horizontal_flip=True)
test_imagedata = ImageDataGenerator(rescale=1. / 255)
training_set = \
    train_imagedata.flow_from_directory('./FolderFrames/MrsPiggyDataset/MrsPiggyTrainingSet'
        , target_size=(64, 64), batch_size=12, class_mode='binary')
val_set = \
    test_imagedata.flow_from_directory('./FolderFrames/MrsPiggyDataset/MrsPiggyValSet'
        , target_size=(64, 64), batch_size=32, class_mode='binary')
history=classifier.fit_generator(training_set, steps_per_epoch=30, epochs=30,
                         validation_data=val_set,
                         validation_steps=30, callbacks=[tf.keras.callbacks.TensorBoard()])

Found 3642 images belonging to 2 classes.
Found 408 images belonging to 2 classes.
Epoch 1/30
30/30 [==============================] - 9s 294ms/step - loss: 0.0889 - accuracy: 0.9750 - val_loss: 0.0929 - val_accuracy: 0.9809
Epoch 2/30
30/30 [==============================] - 8s 282ms/step - loss: 0.0269 - accuracy: 0.9944 - val_loss: 0.0683 - val_accuracy: 0.9883
Epoch 3/30
30/30 [==============================] - 9s 284ms/step - loss: 0.0167 - accuracy: 0.9944 - val_loss: 0.0857 - val_accuracy: 0.9831
Epoch 4/30
30/30 [==============================] - 9s 289ms/step - loss: 0.0354 - accuracy: 0.9889 - val_loss: 0.0838 - val_accuracy: 0.9788
Epoch 5/30
30/30 [==============================] - 9s 303ms/step - loss: 0.0125 - accuracy: 0.9944 - val_loss: 0.0840 - val_accuracy: 0.9799
Epoch 6/30
30/30 [==============================] - 9s 286ms/step - loss: 0.0338 - accuracy: 0.9917 - val_loss: 0.0667 - val_accuracy: 0.9852
Epoch 7/30
30/30 [==============================] - 9s 288ms/step

In [20]:
# serialize model to JSON
model_json = classifier.to_json()
with open("./FolderFrames/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classifier.save_weights("./FolderFrames/model.h5")
print("Saved model to disk")
classifier.summary()

Saved model to disk
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 

In [5]:
#evaluate model
from sklearn.metrics import roc_curve
from tensorflow.keras.models import model_from_json
from PIL import Image
import glob
import cv2 as cv2

test_set = []
for filename in glob.glob('./FolderFrames/MrsPiggyDataset/MrsPiggyTestSet/*.jpg'): #assuming gif
    im = np.asarray(Image.open(filename))
    im = cv2.resize(im, dsize=(64, 64), interpolation=cv2.INTER_CUBIC)
    im = np.expand_dims(im, axis=0)
    test_set.append(im)

# load json and create model
json_file = open('./FolderFrames/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./FolderFrames/model.h5")
print("Loaded model from disk")


y_pred_keras = loaded_model.predict(test_set).ravel()
#fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred_keras)

Loaded model from disk
